In [143]:
from datasets import load_dataset, load_metric
import transformers
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

In [144]:
# Setting random seed for reproducibility
transformers.set_seed(42)

# Loading the dataset
data = pd.read_csv('../../data/interim/training_data.csv', index_col=False)
data = data[:100]
metric = load_metric("sacrebleu")
print(data.isna().sum())

reference      0
translation    0
dtype: int64


In [145]:
# selecting model checkpoint
model_checkpoint = "t5-small"

In [146]:
from transformers import AutoTokenizer

# we will use autotokenizer for this purpose
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [147]:
prefix = "Make this text less toxic:"

In [148]:
import pyarrow as pa
from datasets import Dataset, DatasetDict

dataset = Dataset(pa.Table.from_pandas(data.reset_index(drop=True)))

In [149]:
train_test_split_dataset = dataset.train_test_split(test_size=0.1, seed=42)

# Split the train set further into training and validation sets
train_val_split = train_test_split_dataset['train'].train_test_split(test_size=0.1, seed=42)

# Combine the splits into a single DatasetDict
final_splits = DatasetDict({
    'train': train_val_split['train'],
    'validation': train_val_split['test'],
    'test': train_test_split_dataset['test']
})

final_splits["train"][:2]

{'reference': ['Hows the water, Violator',
  'well burn the rest of the way into battle.'],
 'translation': ['whats the water, Violent',
  'We will blaze a path into battle for others to follow.']}

In [150]:
max_input_length = 256
max_target_length = 256

def preprocess_function(examples):
    inputs = [prefix + ex for ex in examples["reference"]]
    targets = [ex for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [151]:
tokenized_datasets = final_splits.map(preprocess_function, batched=True)
tokenized_datasets['train'][0]

Map:   0%|          | 0/81 [00:00<?, ? examples/s]

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

{'reference': 'Hows the water, Violator',
 'translation': 'whats the water, Violent',
 'input_ids': [1796,
  48,
  1499,
  705,
  12068,
  10,
  7825,
  7,
  8,
  387,
  6,
  1813,
  32,
  40,
  1016,
  1],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [125, 7, 8, 387, 6, 1813, 32, 6987, 1]}

In [152]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
# create a model for the pretrained model
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [153]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load the model and tokenizer
toxic_val_model_name = "unitary/toxic-bert"
toxic_val_tokenizer = AutoTokenizer.from_pretrained(toxic_val_model_name)
toxic_val_model = AutoModelForSequenceClassification.from_pretrained(toxic_val_model_name)

# Function to predict toxicity
def predict_toxicity(text):
    inputs = toxic_val_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = toxic_val_model(**inputs)
    predictions = torch.sigmoid(outputs.logits)
    # Convert predictions to binary 0 or 1 (customize threshold as needed)
    predicted_labels = predictions.float()
    return predicted_labels


In [154]:
# defining the parameters for training
batch_size = 32
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"text-detoxifier-t5-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
)

In [155]:
# instead of writing collate_fn function we will use DataCollatorForSeq2Seq
# simliarly it implements the batch creation for training
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [180]:
import numpy as np


# simple postprocessing for text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


# compute metrics function to pass to trainer
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    (
        toxic_sum,
        severe_toxic_sum,
        obscene_sum,
        threat_sum,
        insult_sum,
        identity_hate_sum,
    ) = (
        0,
        0,
        0,
        0,
        0,
        0,
    )
    for text in decoded_preds:
        toxic, severe_toxic, obscene, threat, insult, identity_hate = tuple(
            *predict_toxicity(text).tolist(),
        )
        toxic_sum += toxic
        severe_toxic_sum += severe_toxic
        obscene_sum += obscene
        threat_sum += threat
        insult_sum += insult
        identity_hate_sum += identity_hate

    result["toxic_average"] = toxic_sum / len(decoded_preds)
    result["severe_toxic_average"] = severe_toxic_sum / len(decoded_preds)
    result["obscene_average"] = obscene_sum / len(decoded_preds)
    result["threat_average"] = threat_sum / len(decoded_preds)
    result["insult_average"] = insult_sum / len(decoded_preds)
    result["identity_hate_average"] = identity_hate_sum / len(decoded_preds)

    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [181]:
# instead of writing train loop we will use Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
trainer.train()

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 1.1509010791778564, 'eval_bleu': 7.7867, 'eval_toxic_average': 0.186, 'eval_severe_toxic_average': 0.0004, 'eval_obscene_average': 0.0101, 'eval_threat_average': 0.0005, 'eval_insult_average': 0.0167, 'eval_identity_hate_average': 0.002, 'eval_runtime': 1.1336, 'eval_samples_per_second': 7.939, 'eval_steps_per_second': 0.882, 'epoch': 1.0}
{'train_runtime': 3.591, 'train_samples_per_second': 22.556, 'train_steps_per_second': 0.835, 'train_loss': 1.0613538424173992, 'epoch': 1.0}


TrainOutput(global_step=3, training_loss=1.0613538424173992, metrics={'train_runtime': 3.591, 'train_samples_per_second': 22.556, 'train_steps_per_second': 0.835, 'train_loss': 1.0613538424173992, 'epoch': 1.0})

In [ ]:
# saving model
trainer.save_model('best')

In [72]:
# loading the model and run inference for it
model = AutoModelForSeq2SeqLM.from_pretrained('best')
model.eval()
model.config.use_cache = False

In [73]:
def translate(model, inference_request, tokenizer=tokenizer):
    input_ids = tokenizer(inference_request, return_tensors="pt").input_ids
    outputs = model.generate(input_ids=input_ids)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True,temperature=0))

In [74]:
inference_request = prefix + 'Why the fuck did it take your stupid ass so long to do it?'
translate(model, inference_request,tokenizer)

to get to the point where to get to the point.


/Users/random/miniconda3/envs/pmldl/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
